In [1]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking import eudx
from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel, quantize_evecs
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model, DeterministicMaximumDirectionGetter
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper

Using TensorFlow backend.


In [2]:
pTrainData_fibrePrediction = 'train_prediction_grid_normalized_dti_cs1_wholebrain.h5'
pTrainData_fibreTracking = 'train_tracking_grid_normalized_dti_cs1_wholebrain.h5'
pTrainInput = 'train_input_normalized_dti_cs1_wholebrain_'
noCrossings = 3

In [3]:
import os
print(os.getcwd())

/home/nico/Code/deepFibreTracking


# Preprocessing

crop multi-shell DWI to single shell data

In [4]:
bvals,bvecs,gtab,dwi,aff,t1,binarymask = dwi_tools.loadHCPData('100307')
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(1000, bvals, bvecs, dwi)

The average b0 image is required for data normalization.

In [5]:
b0_idx = bvals < 10 # 10/02/18 NH: changed to work with HCP data
b0 = dwi[..., b0_idx].mean(axis=3)

In [6]:
dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)

# Tractography

In this section we'll generate streamlines using different approaches. This is gonna be the foundation for the evaluation of our method. We'll also employ simulated as well as curated data for our analysis.

In [7]:
# roi_idx = (slice(20, 50), slice(55, 85), slice(38, 39)) #  splenium of the corpus callosum
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask
ccmask, options = nrrd.read('100307/100307-ccSegmentation.nrrd')
ccseeds = seeds_from_mask(ccmask, affine=aff)
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)
validationSeeds = ccseeds[45:48]
rndseeds = random_seeds_from_mask(binarymask, seeds_count=4000, seed_count_per_voxel=False, affine=aff)

In [8]:
dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)

Percentage erroneous voxels: 4.07


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:72: RuntimeWarning: divide by zero encountered in true_divide
  weights_normed = (weights / b0)
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:72: RuntimeWarning: invalid value encountered in true_divide
  weights_normed = (weights / b0)


In [22]:
### not necessary anymore
#import importlib
#importlib.reload(dwi_tools)
#import src.dwi_tools as dwi_tools
data_sh = dwi_tools.get_spherical_harmonics_coefficients(bvals=bvals_singleShell,bvecs=bvecs_singleShell,sh_order=4, dwi=dwi_singleShell_norm, b0 = 0)

## Import preprocessed data

In [ ]:
import nibabel as nib
f = nib.streamlines.load('100307_new/tracts.trk', lazy_load=False)

In [ ]:
import nibabel as nib
f = nib.streamlines.load('/home/nico/tract.trk', lazy_load=False)

In [ ]:
sl = f.streamlines

## Single Tensor Model

raw dwi data

In [17]:
import dipy.reconst.dti as dti
start_time = time.time()
dti_model = dti.TensorModel(gtab_singleShell)
dti_fit = dti_model.fit(dwi_singleShell_norm, mask=binarymask)
dti_fit_odf = dti_fit.odf(sphere = default_sphere)
dg = DeterministicMaximumDirectionGetter
dg = dg.from_pmf(dti_fit_odf, max_angle=30., sphere=default_sphere)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

Runtime 167.81133651733398s


In [ ]:
sphere = get_sphere('symmetric724')
start_time = time.time()
dtipeaks = peaks_from_model(model=dti_model,
                            data=dwi_singleShell_norm,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            mask=binarymask,
                            return_odf=False,
                            parallel=True,
                            normalize_peaks=False)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s')

In [ ]:
dtiPeakDirs = dtipeaks.peak_dirs[:,:,:,0,:]

spherical harmonics

In [ ]:
from dipy.data import default_sphere
from dipy.direction import DeterministicMaximumDirectionGetter

start_time = time.time()

csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, None, sh_order=6)
csd_fit = csd_model.fit(dwi_singleShell_norm, mask=binarymask)


dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                             max_angle=30.,
                                                             sphere=default_sphere)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + 's')

track and visualize streamlines

In [21]:
from dipy.tracking.local import BinaryTissueClassifier
classifier = ThresholdTissueClassifier(dti_fit.fa, .15)
streamline_generator = LocalTracking(dg, classifier, wholebrainseeds, aff, step_size=1.)
streamlines = list(streamline_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 40)

In [ ]:
from dipy.tracking.local import BinaryTissueClassifier

#binary_classifier = BinaryTissueClassifier(binarymask == 1)
classifier = ThresholdTissueClassifier(dti_fit.fa, .15) # used to be 0.01 10/16/18
streamline_generator = LocalTracking(dg, classifier, wholebrainseeds, aff, step_size=.1)
streamlines = list(streamline_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 40)

In [20]:
import importlib
importlib.reload(dwi_tools)
import src.dwi_tools as dwi_tools
from dipy.tracking.streamline import transform_streamlines
streamlines_imageCS = transform_streamlines(streamlines_filtered,np.linalg.inv(aff)) # project streamlines from RAS into image (voxel) coordinate system
dwi_tools.visStreamlines(streamlines_imageCS,t1)

/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [ ]:
classifier = ThresholdTissueClassifier(dti_fit.fa, .2) # used to be 0.01 10/16/18
streamline_generator = LocalTracking(dg, classifier, validationSeeds, aff, step_size=.1)
streamlines_val = list(streamline_generator)
streamlines_val_filtered = dwi_tools.filterStreamlinesByLength(streamlines_val, 40)
streamlines_val_imageCS = transform_streamlines(streamlines_val_filtered,np.linalg.inv(aff)) # project streamlines from RAS into image (voxel) coordinate system
np.save('streamlines_validation.npy', streamlines_val_imageCS)

## Q-ball Reconstruction

In [ ]:
csamodel = CsaOdfModel(gtab, 4)
sphere = get_sphere('symmetric724')
start_time = time.time()
csapeaks = peaks_from_model(model=csamodel,
                            data=dwi,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            mask=binarymask,
                            return_odf=False,
                            parallel=True,
                            normalize_peaks=False)

GFA = csapeaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

tracking

In [ ]:
classifier = ThresholdTissueClassifier(dtipeaks.gfa, .25)
streamlines_generator = LocalTracking(csapeaks, classifier, seeds, np.identity(4), step_size=.5)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

Visualisation

In [ ]:
dwi_tools.visStreamlines(streamlines_filtered_dg,t1)
#dwi_tools.visStreamlines(streamlines_filtered,t1,vol_slice_idx=76)

## Constrained Spherical Deconvolution

Use in case of b = 2,500 – 3,000 s/mm² data

In [ ]:
response, ratio = auto_response(gtab_singleShell, dwi_singleShell, roi_radius=10, fa_thr=0.7)

In [ ]:
csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, response)
sphere = get_sphere('symmetric724')
start_time = time.time()
csd_peaks = peaks_from_model(model=csd_model,
                             data=dwi_singleShell,
                             sphere=sphere,
                             mask=binarymask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)
GFA = csd_peaks.gfa
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s / GFA.shape (%d, %d, %d)' % GFA.shape)

In [ ]:
csd_fit = csd_model.fit(dwi_singleShell, mask=binarymask)

from dipy.direction import DeterministicMaximumDirectionGetter

detmax_dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                             max_angle=30.,
                                                             sphere=sphere)
streamlines_generator = LocalTracking(detmax_dg, classifier, ccseeds, aff, step_size=.1)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered_dg = dwi_tools.filterStreamlinesByLength(streamlines, 50)

In [ ]:
classifier = ThresholdTissueClassifier(csd_peaks.gfa, .2)
streamlines_generator = LocalTracking(csd_peaks, classifier, ccseeds, aff, step_size=.1)
streamlines = Streamlines(streamlines_generator)
streamlines_filtered = dwi_tools.filterStreamlinesByLength(streamlines, 50)

### Store intermediate data

In [13]:
np.save(pTrainInput + "_sl_filt.npy",streamlines_filtered)
np.save(pTrainInput + "_seeds.npy",ccseeds)

# Generate Training data

In [ ]:
del streamlines_filtered

In [10]:
streamlines_filtered = np.load(pTrainInput + "_sl_filt.npy") # RAS cs
#streamlines_filtered = transform_streamlines(streamlines_imageCS, aff) # project streamlines from image (voxel) into RAS coordinate system

In [25]:
noX = 1
noY = 1
noZ = 1
coordinateScaling = 1

In [26]:
sl_r = np.random.choice(streamlines_filtered,30000, replace=False)

In [28]:
import importlib
importlib.reload(dwi_tools)
import src.dwi_tools as dwi_tools

#rawData = dwi_B0normalized
#rawData = dtiPeakDirs
#rawData = dti_fit.quadratic_form.reshape([145,174,145,9])
#rawData = dti_fit_odf
#rawData = csd_fit.shm_coeff # spherical harmonics
rawData = data_sh
#rawData = tensors
#rawData = dtiPeakDirs

start_time = time.time()
train_DWI,train_prevDirection, train_nextDirection = dwi_tools.generateTrainingData(sl_r, rawData, affine=aff, noX=noX,noY=noY,noZ=noZ,coordinateScaling=coordinateScaling,distToNeighbours=1, noCrossings = noCrossings)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

Processing streamlines
0/30000


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:394: RuntimeWarning: invalid value encountered in true_divide
  directionToPreviousStreamlinePoint[ctr,] = np.nan_to_num(directionToPreviousStreamlinePoint[ctr,] / np.sqrt(np.sum(directionToPreviousStreamlinePoint[ctr,] ** 2))) # unit vector
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:391: RuntimeWarning: invalid value encountered in true_divide
  directionToNextStreamlinePoint[ctr,] = np.nan_to_num(directionToNextStreamlinePoint[ctr,] / np.sqrt(np.sum(directionToNextStreamlinePoint[ctr,] ** 2))) # unit vector


100/30000
200/30000
300/30000
400/30000
500/30000
600/30000
700/30000
800/30000
900/30000
1000/30000
1100/30000
1200/30000
1300/30000
1400/30000
1500/30000
1600/30000
1700/30000
1800/30000
1900/30000
2000/30000
2100/30000
2200/30000
2300/30000
2400/30000
2500/30000
2600/30000
2700/30000
2800/30000
2900/30000
3000/30000
3100/30000
3200/30000
3300/30000
3400/30000
3500/30000
3600/30000
3700/30000
3800/30000
3900/30000
4000/30000
4100/30000
4200/30000
4300/30000
4400/30000
4500/30000
4600/30000
4700/30000
4800/30000
4900/30000
5000/30000
5100/30000
5200/30000
5300/30000
5400/30000
5500/30000
5600/30000
5700/30000
5800/30000
5900/30000
6000/30000
6100/30000
6200/30000
6300/30000
6400/30000
6500/30000
6600/30000
6700/30000
6800/30000
6900/30000
7000/30000
7100/30000
7200/30000
7300/30000
7400/30000
7500/30000
7600/30000
7700/30000
7800/30000
7900/30000
8000/30000
8100/30000
8200/30000
8300/30000
8400/30000
8500/30000
8600/30000
8700/30000
8800/30000
8900/30000
9000/30000
9100/30000
9200/300

In [29]:
pTrainData_fibrePrediction = '102218_train_OLDsh4_step10_wholeBrain_b1k_1x1.h5'

In [30]:
with h5py.File(pTrainData_fibrePrediction,"w") as f:
    f.create_dataset('train_DWI',data=train_DWI)
    f.create_dataset('train_curPosition',data=train_prevDirection)   
#   f.create_dataset('train_LikelyFibreDirections',data=train_LikelyFibreDirections)   
    f.create_dataset('train_NextFibreDirection',data=train_nextDirection)   